# Dependencias

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram, linkage
from pathlib import Path

# Constantes

In [ ]:
PATH_PROJECT_DATA = Path('../data')

# Lectura de los datos

In [ ]:
df_players_cleaned = pd.read_csv(PATH_PROJECT_DATA / 'players_cleaned.csv')
df_matches = pd.read_csv(PATH_PROJECT_DATA / 'matches_grand_slam_cleaned.csv')

# Transformamos las columnas a numerico

Clustering jerárquico solo con valores numéricos

In [ ]:
df_players_cleaned['player_victory'] = (
    pd.get_dummies(df_players_cleaned['player_victory'],
                   drop_first=True)
)

# Agrupamos variables para cada jugador

Vamos a enfocar los clustering para que un jugador tenga todas sus estadísticas agregadas (victorias totales, puntos totales, etc)

In [ ]:
df_players_cleaned_grouped = (
    df_players_cleaned
    .drop(columns=['year', 'tournament', 'seed'])
    .groupby('player_name')
    .sum()
    .reset_index()
)

# Seleccionamos los datos que vamos a utilizar para hacer el clustering

Definimos número de jugadores en `TOP_N` y si queremos ver el top de tenistas o una muestra aleatoria de ese tamaño. 

Esto lo hacemos porque no tiene sentido visualizar los miles de tenistas a la vez para un cluster jerárquico.

In [ ]:
TOP_N = 25
plot_top_players = True

if plot_top_players:
    X = df_players_cleaned_grouped.nlargest(TOP_N, columns='player_victory')
else:
    X = df_players_cleaned_grouped

# Clusterización Jerárquica

Sale bastante chulo, Federer es el tenísta único y le siguen Nadal y Djokovic como siguientes más unicos (y parecidos entre ellos).

In [ ]:
linked = linkage(X.drop(columns='player_name'), 'single')

plt.figure(figsize=(20, 14))
dendrogram(linked,
           orientation='top',
           labels=X['player_name'].tolist())
plt.show()

# KMeans clustering

Utilizamos el Kelbow para ver qué número de variables es el más óptimo para llevar a cabo el clustering

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
from scipy.spatial.distance import cdist

scaler = preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(df_players_cleaned_grouped
                                .drop(columns='player_name'))

# k means determine k
distortions = []
K = range(1,10)
for k in K:
    kmeans = KMeans(n_clusters=k).fit(X_scaled)
    distortions.append(sum(np.min(cdist(X_scaled,
                                        kmeans.cluster_centers_,
                                        'euclidean'), axis=1)) / X_scaled.shape[0])

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

# Número óptimo de clusters: 5

In [ ]:
kmeans = KMeans(random_state=8,
                init='random',
                n_clusters=5,
                n_init=10).fit(X_scaled)

df_players_cleaned_grouped['kmeans'] = kmeans.predict(X_scaled)

# Agrupamos por cluster asignado y vemos numero de victorias

Vemos que hay clusters donde hay muchas más victorias que la media y clusters que tienen muchas menos, no tiene mala pinta.

La media de victorias es de 13.14 por jugador.


In [ ]:
(df_players_cleaned_grouped
 .drop(columns=['kmeans'])
 .merge(df_players_cleaned_grouped[['player_name', 'kmeans']],
        on='player_name',
        how='inner')
 .groupby('kmeans')
 .player_victory
 .agg(['mean', 'count'])
 .reset_index())

## Viendo el top de tenistas vemos como a los primeros cracks los mete en el mismo clustering y viendo a los peores pasa exactamente lo mismo

In [ ]:
USE_LARGESTS = True

if USE_LARGESTS:
    display(df_players_cleaned_grouped
            [['player_name', 'player_victory', 'kmeans']]
            .nlargest(TOP_N, columns='player_victory')
            .groupby('player_name')
            .agg(dict(kmeans=pd.Series.mode, player_victory='first'))
            .reset_index()
            .sort_values(by='player_victory', ascending=False))
else:
    display(df_players_cleaned_grouped
            [['player_name', 'player_victory', 'kmeans']]
            .nsmallest(TOP_N, columns='player_victory')
            .groupby('player_name')
            .agg(dict(kmeans=pd.Series.mode, player_victory='first'))
            .reset_index()
            .sort_values(by='player_victory', ascending=False))